In [1]:
import pandas as pd
import numpy as npv

In [2]:
df = pd.read_csv("GroceryStoreDataSet.csv",delimiter=',')
df.head()

,"MILK,BREAD,BISCUIT"
0,"BREAD,MILK,BISCUIT,CORNFLAKES"
1,"BREAD,TEA,BOURNVITA"
2,"JAM,MAGGI,BREAD,MILK"
3,"MAGGI,TEA,BISCUIT"
4,"BREAD,TEA,BOURNVITA"


In [7]:
l=[]

In [8]:
for i in range(0,len(df['ITEMS'])):
    l.append(df['ITEMS'][i].split(","))

KeyError: 'ITEMS'

In [ ]:
l

In [ ]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [ ]:
createC1(l)

In [ ]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can] = 1
                else: ssCnt[can] += 1
                    
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        supportData[key] = support
        if support >= minSupport:
            retList.insert(0,key)
    return retList,supportData

In [ ]:
dataSet = l
dataSet

In [ ]:
C1 = createC1(dataSet)
C1

In [ ]:
D = list(map(set,dataSet))
D

In [ ]:
L1,suppDat0 = scanD(D,C1,0.1)
L1

In [ ]:
suppDat0

In [ ]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) 
    return retList

In [ ]:
def apriori(dataSet, minSupport):
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))
    L1, supportData = scanD(D,C1,minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2],k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [ ]:
def generateRules(L, supportData, minConf):
    bigRuleList = []
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i>1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList,minConf)
            else:
                prunedH = calcConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList

In [ ]:
def calcConf(freqSet, H, supportData,brl,minConf):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet-conseq, " --> ", conseq, "conf: ", conf)
            brl.append((freqSet-conseq , conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [ ]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf):
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H,m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData,brl,minConf)
        
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData,brl,minConf)

In [ ]:
L, suppData = apriori(dataSet,minSupport=0.1)

In [ ]:
rules = generateRules(L, suppData, minConf = 0.4)

In [ ]:
L